# Library

In [28]:
import psycopg2
import pandas as pd
from decouple import config
from bokeh.io import curdoc
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, RangeTool, NumeralTickFormatter, Range1d, LabelSet, HoverTool
from bokeh.themes import Theme
from bokeh.embed import components
from bokeh.transform import cumsum
from bokeh.layouts import layout
from bokeh.models.widgets import Tabs, Panel

In [29]:
# Styles for Bokeh Plots
THEME = Theme(json={
    'attrs': {
        'Figure': {
#             'sizing_mode': 'stretch_width',
#             'plot_height': 500,
            'background_fill_color': '#FFFFFF',
            'border_fill_color': '#FFFFFF',
            'outline_line_color': '#FFFFFF',
        },
        'Axis': {
            'major_label_text_font_size': '12pt',
            'minor_tick_line_color': "white",
            'axis_label_text_font_size': '14pt',
            'axis_label_text_font_style': 'bold',
            'axis_label_text_font_style': 'italic'
        },
        'Grid': {
            'grid_line_color': None
        },
        'Line': {
            'line_color': '#000000',
            'line_width': 2,
        },
        'Bar': {
            'fill_color': '#000000'
        },
        'Title': {
            'text_color': "#000000",
            'text_font_size': '16pt',
            'text_font_style': 'bold'
        }
    }
})
doc = curdoc()
doc.theme = THEME

# Colors for charts
PRIMARY_COLOR = '#000000'
SECONDARY_COLOR = '#000000'
GRAY = '#A9A9A9'
COLORS = ['#4101f5', '#21e68c', '#b22c99',
          '#8c1932', '#509bf5', '#fae62c', '#f59b22', '#f5729f', '#ff4935']
output_notebook()

Loading BokehJS ...

# Connect to Database

In [13]:
# Connect to DB
connection = psycopg2.connect(
    host = config('DB_HOST'),
    port = 5432,
    user = config('DB_USERNAME'),
    password = config('DB_PASSWORD'),
    database='postgres'
    )
cursor=connection.cursor()

# Get Max Date in DB

In [14]:
q = """
SELECT
    MAX(DATE(date))
FROM
    public.webapp_dailycoin;
"""
cursor.execute(q)
connection.commit()
max_date = cursor.fetchall()[0][0]

In [16]:
str(max_date)

'2021-09-26'

# Get Latest Data

In [24]:
q = f"""
SELECT
    *
FROM
    public.webapp_dailycoin
WHERE
    DATE(date) = '{str(max_date)}'
"""
latest_data = pd.read_sql_query(q, connection)

In [32]:
latest_data.sort_values(by='tweet_volume', inplace=True)

# Tweet Volume

In [34]:
source = ColumnDataSource(latest_data)
p = figure(
    y_range=latest_data['coin'],
    plot_width=800,
    plot_height=500,
    title=f"Tweet Volume by Coin on {str(max_date)}",
    toolbar_location=None
)
p.hbar(y='coin', right='tweet_volume', color='#000000', height=.5, source=source)
p.x_range = Range1d(0, latest_data['tweet_volume'].max()*1.05)
p.xaxis.formatter = NumeralTickFormatter(format='0,0a')
p.xaxis.axis_label = "Tweet Volume"
show(p)